# LangChainの応用：create_agentで作成するWeb検索連動チャットボット

## 1.準備

In [ ]:
# 必要なモジュールをインポート
import uuid # UUID（ランダムなID）生成用
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver # メモリ保存用
from langchain_core.messages import AIMessageChunk
from IPython.display import Image, display

# 環境変数の読み込み
load_dotenv()

# モデル名
MODEL_NAME = "gpt-5-mini"

## 2.ツールの準備

In [ ]:
from langchain_tavily import TavilySearch
from langchain_core.tools import tool

# ツールの初期化
tavily_search = TavilySearch(
    max_results=2,                 # 取得する検索結果の数
    search_depth="basic",          # "basic" (高速) か "advanced" (高品質)
    include_answer=False,          # Tavilyが生成した短い回答を含めない
    include_raw_content=False,     # HTMLの生コンテンツを含めるか（トークン消費増に注意）
    include_images=False,          # 画像URLを含めるか
    # include_domains=["go.jp"],   # 特定のドメインのみ検索する場合
    # exclude_domains=["wikipedia.org"] # 特定のドメインを除外する場合
)

# Tavilyの検索結果だけを、LLMが読みやすい形に整形する関数
def format_tavily_results(tavily_response: dict) -> str:
    results = tavily_response.get("results", [])
    if not results:
        return "（検索結果なし）"

    lines = []
    for i, r in enumerate(results, 1):
        title = r.get("title", "")
        content = r.get("content", "")
        url = r.get("url", "")
        lines.append(f"[{i}] {title}\n{content}\nsource: {url}")
    return "\n\n".join(lines)

# TavilySearchの生レスポンス(dict/JSON)を整形して返すラッパーTool
@tool
def tavily_search_formatted(query: str) -> str:
    """Web検索（Tavily）。上位結果を整形して返す。"""
    tavily_response = tavily_search.invoke({"query": query})
    return format_tavily_results(tavily_response)

# create_agent の tools に渡す
tools = [tavily_search_formatted]

## 3.エージェントの作成

In [ ]:
# 2. モデルの初期化
model = ChatOpenAI(model=MODEL_NAME)

# 3. 記憶領域の初期化
checkpointer = InMemorySaver()

prompt_text = """
ツール呼び出しを行った後に回答する場合は以下を行ってください。
・回答の末尾に参照したsource番号（例：[1][2]）を付ける
・最後にsource番号とurlを箇条書きで列挙する
"""

# 4. エージェントの作成（ツールを登録）
# create_agentが自動的にツール呼び出しの判断ロジックを構築します
search_agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=checkpointer,
    system_prompt=prompt_text,
)

# グラフの可視化
display(Image(search_agent.get_graph().draw_mermaid_png()))

## 4.メインループ

In [ ]:
# ランダムなUUIDを生成して thread_id に設定
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

while True:
    user_input = input("メッセージを入力:")
    if user_input.strip() == "":
        break
    print(f"質問：{user_input}")

    # 検索中フラグ
    is_spinning = False

    # エージェントを実行し、応答をストリーミング表示
    for chunk, metadata in search_agent.stream(
        {"messages": [{"role": "human", "content": user_input}]},
        config=config,
        stream_mode="messages"
    ):
        # 1. AIメッセージ（LLMの出力）のみを対象とする
        if isinstance(chunk, AIMessageChunk):
            # 2. ツール呼び出しの定義（JSON生成）中は表示しない
            if chunk.tool_call_chunks:
                if chunk.tool_call_chunks[-1]["name"] != None:
                    print(".", end="", flush=True) # 考え中の表現
                    is_spinning = True
                continue

            # 3. コンテンツ（最終回答のテキスト）が含まれている場合のみ表示
            if chunk.content:
                if is_spinning:
                    print() # 初回のみ改行を入れる
                    is_spinning = False
                print(chunk.content, end="", flush=True)
    print() # 改行
            
print("\n---ご利用ありがとうございました！---")